# gem_identifier_model

In [1]:
!pip install opendatasets -q
!pip install tensorflow matplotlib seaborn scikit-learn -q


In [2]:

# import necessary libraries
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import time

import opendatasets as od

In [3]:
od.download("https://www.kaggle.com/datasets/lsind18/gemstones-images")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: binojmadhuranga
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/lsind18/gemstones-images


100%|██████████| 55.2M/55.2M [00:00<00:00, 1.26GB/s]

In [4]:
import os

base_path = "/content/gemstones-images"
os.listdir(base_path)


['test', 'train']

**Load Training Dataset in TensorFlow**

In [14]:
import tensorflow as tf

img_size = (224, 224)
batch_size = 32

train_dir = "/content/gemstones-images/train"

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    validation_split=0.2,   # 20% for validation
    subset="training",
    seed=42,
    image_size=img_size,
    batch_size=batch_size
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    validation_split=0.2,
    subset="validation",
    seed=42,
    image_size=img_size,
    batch_size=batch_size
)


Found 2856 files belonging to 87 classes.
Using 2285 files for training.
Found 2856 files belonging to 87 classes.
Using 571 files for validation.


In [19]:
tf.keras.backend.clear_session()


**Build a Transfer Learning Model**

In [20]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras import layers, models

num_classes = len(train_ds.class_names)

base_model = EfficientNetB0(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # freeze base model initially

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.2),
    layers.Dense(num_classes, activation="softmax")
])

model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ efficientnetb0 (Functional)     │ (None, 7, 7, 1280)     │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 87)             │       111,447 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,161,018 (15.87 MB)

 Trainable params: 111,447 (435.34 KB)

 Non-trainable params: 4,049,571 (15.45 MB)

**Train the model**

In [21]:
test_dir = "/content/gemstones-images/test"

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,
    seed=42,
    image_size=img_size,
    batch_size=batch_size
)


Found 363 files belonging to 87 classes.


In [22]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10
)


Epoch 1/10
72/72 ━━━━━━━━━━━━━━━━━━━━ 237s 3s/step - accuracy: 0.1066 - loss: 4.1397 - val_accuracy: 0.3783 - val_loss: 2.7949
Epoch 2/10
72/72 ━━━━━━━━━━━━━━━━━━━━ 182s 3s/step - accuracy: 0.5230 - loss: 2.3842 - val_accuracy: 0.5359 - val_loss: 2.0422
Epoch 3/10
72/72 ━━━━━━━━━━━━━━━━━━━━ 199s 2s/step - accuracy: 0.6718 - loss: 1.6427 - val_accuracy: 0.6077 - val_loss: 1.7019
Epoch 4/10
72/72 ━━━━━━━━━━━━━━━━━━━━ 209s 3s/step - accuracy: 0.7649 - loss: 1.2359 - val_accuracy: 0.6497 - val_loss: 1.4965
Epoch 5/10
72/72 ━━━━━━━━━━━━━━━━━━━━ 180s 3s/step - accuracy: 0.8093 - loss: 1.0068 - val_accuracy: 0.6725 - val_loss: 1.3610
Epoch 6/10
72/72 ━━━━━━━━━━━━━━━━━━━━ 186s 3s/step - accuracy: 0.8416 - loss: 0.8550 - val_accuracy: 0.6935 - val_loss: 1.2721
Epoch 7/10
72/72 ━━━━━━━━━━━━━━━━━━━━ 204s 3s/step - accuracy: 0.8628 - loss: 0.7387 - val_accuracy: 0.7005 - val_loss: 1.1947
Epoch 8/10
72/72 ━━━━━━━━━━━━━━━━━━━━ 195s 3s/step - accuracy: 0.8884 - loss: 0.6309 - val_accuracy: 0.7075 - v

SAVE

In [ ]:
model.save("gem_identifier_model.h5")


In [ ]:
model.save("gem_identifier_model.keras")